# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [136]:
## Connect to DB server on AWS
import config
import mysql.connector 

cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = "Yelp_API"
)

In [137]:
cursor = cnx.cursor()


In [3]:
## Create new DB 
from mysql.connector import errorcode

db_name = "Yelp_API"

def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [4]:
# Create a table for the Businesses
DB_NAME = 'Yelp_API'

TABLES = {}
TABLES['businesses'] = (
    "CREATE TABLE businesses ("
    "  business_id varchar(255) NOT NULL UNIQUE PRIMARY KEY,"
    "  name varchar(255) NOT NULL,"
    "  review_count int NOT NULL,"
    "  price varchar(5) NOT NULL,"
    "  rating int NOT NULL"
    ") ENGINE=InnoDB")


In [5]:
# Create a table for the reviews
TABLES['reviews'] = (
    "CREATE TABLE reviews ("
    "  review_id int NOT NULL PRIMARY KEY,"
    "  rating int NOT NULL,"
    "  date date NOT NULL,"
    "  text varchar(255) NOT NULL,"
    "  business_id varchar(255) NOT NULL,"
    "  FOREIGN KEY (business_id) REFERENCES businesses (business_id)"
    ") ENGINE=InnoDB")

In [6]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


Creating table businesses: OK
Creating table reviews: OK


## Part 2: Create ETL pipeline for the business data from the API

In [176]:
import json 
import requests
import pandas as pd

api_key = 'pDvSQP6jodi0QbUqS_gHcli4oUwFYxhsubzLR9t_K6wGo6iz0jCmuV3ob5wjzLmCrZss9nCjf0hZOvHCpSXbGBtHwHNSzGEqk5irVO1SXslzhvoYuMwayekbi_uyXXYx'


In [19]:
# Write a function to make a call to the yelp API



def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data


import time

term = 'pizza'
location = 'Harlem NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50,
              
             }
# df = pd.DataFrame(yelp_call(url_params, api_key))
# print(len(df))


In [30]:
term = 'pizza'
location = 'Harlem NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50,
              
             }
url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'Bearer {}'.format(api_key)}
response = requests.get(url, headers=headers, params=url_params)

def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    

    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
        query(insert_query, business_list(parsed_results))
        time.sleep(1) #Wait a second
        cur += 50
  


In [31]:
all_results(url_params, api_key)

703 total matches found.


In [42]:
print(len(df))


50


In [ ]:
for result in results:
    if 'price' not in result.keys():
        result['price']= None
    parsed_results.append((result['id'], result['name'], result['price']))

In [22]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB
def parse_results(results):
    
    keys = ['id', 'name', 'review_count', 'rating', 'price']
    parsed_results = []
    for result in results:
        parsed_result = {key: (result[key] if key in result.keys() else 0) 
                         for key in keys}
        parsed_results.append(parsed_result)
    return parsed_results

parsed_results = parse_results(yelp_call(url_params, api_key))
parsed_results

[{'id': 'cqDVdoLA_36vpz2qxWdBqA',
  'name': "Uncle Tony's Pizza",
  'review_count': 94,
  'rating': 4.0,
  'price': '$$'},
 {'id': 'NPnWsPxOpG91GHx74S27Uw',
  'name': 'Sottocasa Pizzeria - Harlem',
  'review_count': 245,
  'rating': 4.5,
  'price': '$$'},
 {'id': 'x4_5vkcFKvaxJuZAJRX_Qw',
  'name': 'Harlem Pizza Co',
  'review_count': 182,
  'rating': 4.0,
  'price': '$$'},
 {'id': 'eIOb3a-4UmGSSztGe9MwPw',
  'name': 'Pizza by Lucilles',
  'review_count': 22,
  'rating': 4.5,
  'price': 0},
 {'id': 'bZZzJ2IJiL-ZAn6qen0-xA',
  'name': 'Pizza Stop',
  'review_count': 51,
  'rating': 4.0,
  'price': '$'},
 {'id': 'gyWc5qJRvu26LVQIShdR4Q',
  'name': 'Babbalucci',
  'review_count': 379,
  'rating': 4.0,
  'price': '$$'},
 {'id': 'hUjy33SLPaSipQxZ1ume4Q',
  'name': "Roland's",
  'review_count': 5,
  'rating': 5.0,
  'price': 0},
 {'id': 'B0qxEXABa8-t9sY4NQRRsQ',
  'name': 'Fumo',
  'review_count': 391,
  'rating': 4.0,
  'price': '$$'},
 {'id': '-HQWFXtDoRwu6D3rvH8nTQ',
  'name': "King's Piz

In [44]:
url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'Bearer {}'.format(api_key)}
response = requests.get(url, headers=headers, params=url_params)
        
# all_results = all_results(parse_results(yelp_call(url_params, api_key)))

In [29]:
# Write a function to take your parsed data and insert it into the DB
def business_list(parsed_results):    
    business_list = []
    for business in parsed_results:
        business_tuple = (business['id'], business['name'], business['review_count'],
                         business['rating'], business['price'])
        business_list.append(business_tuple)
    return business_list


business_list(parsed_results)

[('cqDVdoLA_36vpz2qxWdBqA', "Uncle Tony's Pizza", 94, 4.0, '$$'),
 ('NPnWsPxOpG91GHx74S27Uw', 'Sottocasa Pizzeria - Harlem', 245, 4.5, '$$'),
 ('x4_5vkcFKvaxJuZAJRX_Qw', 'Harlem Pizza Co', 182, 4.0, '$$'),
 ('eIOb3a-4UmGSSztGe9MwPw', 'Pizza by Lucilles', 22, 4.5, 0),
 ('bZZzJ2IJiL-ZAn6qen0-xA', 'Pizza Stop', 51, 4.0, '$'),
 ('gyWc5qJRvu26LVQIShdR4Q', 'Babbalucci', 379, 4.0, '$$'),
 ('hUjy33SLPaSipQxZ1ume4Q', "Roland's", 5, 5.0, 0),
 ('B0qxEXABa8-t9sY4NQRRsQ', 'Fumo', 391, 4.0, '$$'),
 ('-HQWFXtDoRwu6D3rvH8nTQ', "King's Pizza of Harlem", 53, 4.0, '$'),
 ('pIrSHmLYW6Bdjiz-HHWd1Q', 'Bettolona', 290, 4.0, '$$'),
 ('OaI0egMsvwAhcJT6FDcfAQ', 'P&M Classic Pizza & Restaurant', 58, 4.0, '$'),
 ('49TSSzBpYcRTfyhbVDZHig', "Olga's pizza", 41, 4.0, '$'),
 ('HXjbgOrN0DyAnfAPvnssSQ', 'Pizza Stop', 114, 3.5, '$'),
 ('wLzJbddDixAEZrF7VT6_hw', 'Bono Trattoria', 289, 4.0, '$$'),
 ('QKFTUMe5dkAhqAqbiuwtWA', 'Uptown Slices', 6, 4.0, 0),
 ('Kh0-HmlgmVUkRxR55_Vn1g', "Patsy's Pizzeria", 817, 4.0, '$$'),
 ('fR

In [26]:
insert_query = """INSERT INTO businesses (business_id, name, review_count, rating, price) 
                VALUES (%s, %s, %s, %s, %s);"""

In [47]:
cursor.executemany(insert_query, business_list)

cnx.commit()

In [25]:
# Write a script that combines the three functions above into a single process.
def query(insert_query, data):
    cursor.executemany(insert_query, data)
    cnx.commit()



In [ ]:
query(insert_query, business_list(parsed_results))

## Part 3: Create ETL pipeline for the restaurant review data from the API

In [142]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

cursor.execute("""
                SELECT 
                    business_id
                FROM
                    businesses
                ;""")
business_ids = cursor.fetchall()
business_ids

[('-593H2__eJeiSBT4pU7Wfw',),
 ('-bOXDOBK1W0ceWEw0-Cjzw',),
 ('-D4aiRE4YIoRBKbcuDD6KA',),
 ('-DHpd5b4c3fX1yIuOyYpzA',),
 ('-gm0mQIBf78CEHVAskLiSA',),
 ('-HQWFXtDoRwu6D3rvH8nTQ',),
 ('-jcSbz_IBk-NC_ZOXh-r6w',),
 ('-Q8eGYTpq28taq7lzCtx2w',),
 ('-VZhEbVm1mBF2tJj4Iz6kA',),
 ('0CMz9dt2nwt35GiaVrTb2Q',),
 ('0dG6Qqw0TZDYInWPOhDBGQ',),
 ('0DjXlV1cI5OG4SpOILl0HA',),
 ('0Ga71v7UsZZlDrbcJNDj_Q',),
 ('0GFs6ZxeZC-d3KY1RWCSrw',),
 ('0GjkxhZe3Ka6P_MOFXxQ_g',),
 ('0Kcuy_vyfWlAetDgdmIy3g',),
 ('0o0s1INi5_85aVpdUuBJew',),
 ('0UEHlHCe4cvPSRw2URvbKQ',),
 ('0UIUQWmwf_yxyygLXPQ0Hw',),
 ('0xn4GFYigvERx0tjtTaOxQ',),
 ('0xS2Jj-RZuFQKgxEWuVxzA',),
 ('10B5dlMaQ_l3m5QrlCPNzA',),
 ('17l54KZTl0yIXWiGxd0JOA',),
 ('1AVoX4q-YRqWTfxVF7tzyQ',),
 ('1bjUc14khru1HXuMUe5-5w',),
 ('1G8aL9TFGZPSr6GooLRQkg',),
 ('1Lsb6F7DprPLabtbMHNURA',),
 ('1NDrUD3rct843ekECS9Dkw',),
 ('1p4dUaLyzIhN5n1lJnHSgA',),
 ('1pAbrX_NPgfU-b4jGBSnOg',),
 ('1TfwXGWxpfcx_XGkJNdK4g',),
 ('1uOB_tWZwvI0_HakXfu5Jg',),
 ('1uQJ10U7g6Erz5yZ0l_g3w',),
 ('29Xq_4p

In [267]:
# write a function that takes a business id 
# and makes a call to the API for reivews

def yelp_call_reviews(api_key, business_id):  
    
    url = f'https://api.yelp.com/v3/businesses/{business_id[0]}/reviews'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers)

    data = response.json()['reviews']

    return data


In [268]:
review_results = yelp_call_reviews(api_key, business_id)


In [270]:
review_results[0]['business_id'] = business_ids[0][0]
review_results

[{'id': 'HCNtsdfXBPEF8zji-yKMTA',
  'url': 'https://www.yelp.com/biz/viand-new-york?adjust_creative=u4cW5BIyc8XIUE3fg5Jsqg&hrid=HCNtsdfXBPEF8zji-yKMTA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=u4cW5BIyc8XIUE3fg5Jsqg',
  'text': "My favorite diner in the UWS. Always wonderful service and the food is always on point.\n\nI normally come for Brunch. Once I've had an excellent pulled pork...",
  'rating': 5,
  'time_created': '2019-10-14 13:42:36',
  'user': {'id': 'eNuIxFh58ygrUiEucKLGQA',
   'profile_url': 'https://www.yelp.com/user_details?userid=eNuIxFh58ygrUiEucKLGQA',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/photo/H8MNcWSrkIcjzYL2BG7wtg/o.jpg',
   'name': 'Nychoopaholic B.'},
  'business_id': '-593H2__eJeiSBT4pU7Wfw'},
 {'id': 'CU6_6GY7uS_eBOkXfQY7Mg',
  'url': 'https://www.yelp.com/biz/viand-new-york?adjust_creative=u4cW5BIyc8XIUE3fg5Jsqg&hrid=CU6_6GY7uS_eBOkXfQY7Mg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=u4cW5BIyc8

In [280]:
# Write a function to parse out the relevant information from the reviews
parsed_reviews = []
def parse_reviews(reviews, business_id):
    keys = ['id', 'rating', 'date', 'text']
    for review in reviews:
        parsed_review = {}
        for key in keys:
#             for i in range(3):
            print(key)
            parsed_review['business_id'] = business_id
            if key in review.keys():
                parsed_review[key] = review[key]
            else:
                parsed_review[key] = 'Null'
                

            parsed_reviews.append(parsed_review)
        return parsed_reviews

# parsed_reviews = parse_reviews(yelp_call_reviews(url_params, api_key))
# parsed_reviews

In [281]:
# write a function to insert the parsed data into the reviews table

def insert_reviews(parse_reviews):
    review_list = []
    for review in parsed_reviews:
        review_tuple = (review['id'], review['rating'], review['date'],
                         review['text'], review['business_id'])
        review_list.append(review_tuple)

    insert_query = """INSERT INTO businesses (review_id, rating, date, text, business_id) 
                VALUES (%s, %s, %s, %s, %s);"""
    cursor.executemany(insert_query, review_list)

In [282]:
# combine the functions above into a single script  

for business_id in business_ids:
    review_response = yelp_call_reviews(api_key, business_id)
    parse_reviews(review_results, business_id)
    insert_reviews(parse_reviews)

id
rating
date
text


InterfaceError: Failed executing the operation; Python type tuple cannot be converted

# Part 4: Write SQL queries that will answer the questions posed. 

In [ ]:
# create connection

In [ ]:
# execute SQL queries

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [ ]:
def parse_results(results):
    
    return parsed_results

In [90]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time



def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
        db_insert(parsed)
        time.sleep(1) #Wait a second
        cur += 50

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

703 total matches found.


NameError: name 'db_insert' is not defined

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```

### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)